### Model Structure
| Layer             |
| ----------------- |
| `Conv2D(16, 3x3)` |
| `Conv2D(16, 3x3)` |
| `MaxPool2D(2x2)`  |
| `Dropout`         |
| `Conv2D(32, 3x3)` |
| `Conv2D(32, 3x3)` |
| `MaxPool2D(4x4)`  |
| `Dropout`         |
| `Flatten`         |
| `Dense(256)`      |
| `Dropout`         |
| `Dense(1)`        |

![BRAAI CNN Model Structure](images/fig-braai.png)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cupy as np

dataset_dir = "/content/drive/MyDrive/braai_cnn/"
dataset_path = dataset_dir + "ztf_dataset_split.npz"
data = np.load(dataset_path)
data

In [3]:
np.show_config()

OS                           : Linux-6.1.123+-x86_64-with-glibc2.35
Python Version               : 3.11.13
CuPy Version                 : 13.3.0
CuPy Platform                : NVIDIA CUDA
NumPy Version                : 2.0.2
SciPy Version                : 1.15.3
Cython Build Version         : 0.29.36
Cython Runtime Version       : 3.0.12
CUDA Root                    : /usr/local/cuda
nvcc PATH                    : /usr/local/cuda/bin/nvcc
CUDA Build Version           : 12060
CUDA Driver Version          : 12040
CUDA Runtime Version         : 12060 (linked to CuPy) / 12050 (locally installed)
CUDA Extra Include Dirs      : ['/usr/local/lib/python3.11/dist-packages/nvidia/cuda_runtime/include']
cuBLAS Version               : (available)
cuFFT Version                : 11203
cuRAND Version               : 10306
cuSOLVER Version             : (11, 6, 3)
cuSPARSE Version             : (available)
NVRTC Version                : (12, 5)
Thrust Version               : 200600
CUB Build Version  

In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jul 16 18:47:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   39C    P0             16W /   72W |     189MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
X = data["X_train"].astype(np.float16)
y = data["y_train"].astype(np.int8)
X.shape, y.shape

((28776, 3, 63, 63), (28776,))

In [6]:
import time
t = 1000 * time.time()
np.random.seed(int(t) % 2**30)

In [7]:
# Layer inteface
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward(self, input):
        pass

    def backward(self, d_out, lr):
        pass

    def get_parameters(self):
        params = []
        if hasattr(self, 'kernels'):
            params.append(self.kernels)
        if hasattr(self, 'biases'):
            params.append(self.biases)
        if hasattr(self, 'weights'):
            params.append(self.weights)
        return params

    def get_gradients(self):
        grads = []
        if hasattr(self, 'kernels_grad'):
            grads.append(self.kernels_grad)
        if hasattr(self, 'biases_grad'):
            grads.append(self.biases_grad)
        if hasattr(self, 'weights_grad'):
            grads.append(self.weights_grad)
        return grads


    def set_parameters(self, params):
        i = 0
        if hasattr(self, 'kernels'):
            self.kernels = params[i]
            i += 1
        if hasattr(self, 'biases'):
            self.biases = params[i]
            i += 1
        if hasattr(self, 'weights'):
            self.weights = params[i]
            i += 1

In [8]:
class Adam:
    def __init__(self, params, lr, beta1, beta2, epsilon):
        self.params = params
        self.m = [np.zeros_like(param) for param in params]
        self.v = [np.zeros_like(param) for param in params]
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.t = 0

    def step(self, grads):
        self.t += 1
        for i in range(len(self.params)):
            g = grads[i]
            self.m[i] = self.beta1 * self.m[i] + (1-self.beta1)*g
            self.v[i] = self.beta2 * self.v[i] + (1-self.beta2)*(g**2)

            m_hat = self.m[i] / (1 - self.beta1**self.t)
            v_hat = self.v[i] / (1 - self.beta2**self.t)

            self.params[i] -= self.lr * (m_hat / (self.epsilon + np.sqrt(v_hat)))

    def collect_params(self):
        return self.params


In [9]:
# Helper functions
from cupy.lib.stride_tricks import sliding_window_view

# Converts NCHW input to column matrix
def im2col(input, KH, KW, stride=1):
      N, C, H, W = input.shape
      OH = (H - KH) // stride + 1
      OW = (W - KW) // stride + 1

      patches = sliding_window_view(input, (KH, KW), axis=(2,3))
      patches = patches[:, :, ::stride, ::stride, :, :]
      cols = patches.transpose(0,2,3,1,4,5).reshape(N*OH*OW, -1)
      return cols, OH, OW

# Reverse of im2col for accumulating gradients to x
def col2im(cols, input_shape, KH, KW, stride=1):
    N, C, H, W = input_shape
    OH = (H - KH) // stride + 1
    OW = (W - KW) // stride + 1

    patches = cols.reshape(N, OH, OW, C, KH, KW).transpose(0, 3, 4, 5, 1, 2)
    x_grad = np.zeros(input_shape, dtype=cols.dtype)

    for i in range(KH):
        for j in range(KW):
            x_grad[:, :, i:i+stride*OH:stride, j:j+stride*OW:stride] += patches[:, :, i, j]
    return x_grad

In [10]:
from cupyx.scipy.signal import correlate2d, convolve2d

class Conv2D(Layer):
    def __init__(self, kernel_size, output_depth):
        self.output_depth = output_depth
        self.kernel_size = kernel_size
        self.initialized = False

    def initialize_layer(self, input_shape):
        N, self.input_depth, H, W = input_shape
        self.KH = self.kernel_size
        self.KW = self.kernel_size
        self.OH = H - self.KH + 1
        self.OW = W - self.KW + 1

        fan_in = self.input_depth * self.KH * self.KW
        self.kernels = np.random.randn(self.output_depth, self.input_depth, self.KH, self.KW) * np.sqrt(2.0 / fan_in).astype(np.float16)
        self.biases = np.random.randn(self.output_depth).astype(np.float16)
        self.initialized = True

    def forward(self, input):
        if not self.initialized:
            self.initialize_layer(input.shape)

        self.input = input
        # 1. im2 col: all patches in one big matrix
        cols = im2col(input, self.KH, self.KW)[0]

        # 2. reshape kernels to 2D
        W_col = self.kernels.reshape(self.output_depth, -1)

        # 3. GEMM
        out_cols = cols @ W_col.T
        out_cols += self.biases

        # 4. Reshape back to OG
        N = input.shape[0]
        self.output = out_cols.reshape(N, self.OH, self.OW, self.output_depth).transpose(0,3,1,2)
        return self.output

    def backward(self, d_out, lr):
        N = d_out.shape[0]

        # 1. flatten d_out to columns
        d_cols = d_out.transpose(0,2,3,1).reshape(-1, self.output_depth)

        # 2. grad kernels
        cols = im2col(self.input, self.KH, self.KW)[0]
        W_col_grad = d_cols.T @ cols
        self.kernels_grad = W_col_grad.reshape(self.kernels.shape) / N

        # 3. grad input cols
        W_col = self.kernels.reshape(self.output_depth, -1)
        cols_grad = d_cols @ W_col

        # 4. col2im to get input gradient
        input_grad = col2im(cols_grad, self.input.shape, self.KH, self.KW)

        # 5) bias grad
        self.biases_grad = d_out.sum(axis=(0,2,3)) / N
        return input_grad

    def get_gradients(self):
      return [self.kernels_grad, self.biases_grad]

/usr/local/lib/python3.11/dist-packages/cupyx/jit/_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')


In [11]:
class ReLU(Layer):
    def forward(self, input):
        self.input = input
        return np.maximum(0, input)

    def backward(self, d_out, lr):
        # Relu prime
        # print(f"{self.__class__.__name__} received grad of shape {d_out.shape}")

        return d_out * (self.input > 0)

In [12]:
class Flatten(Layer):
    def forward(self, input):
        self.input_shape = input.shape # (N, C, H, W)
        return input.reshape(self.input_shape[0], -1) # (N, C*H*W)

    def backward(self, d_out, lr=None):
        # print(f"{self.__class__.__name__} received grad of shape {d_out.shape}")

        return d_out.reshape(self.input_shape)

In [13]:
class MaxPool2D(Layer):
    def __init__(self, pool_size=2, stride=2):
        self.pool_size = pool_size
        self.stride = stride

    def forward(self, input):
        self.input = input
        N, C, H, W = input.shape

        patches, OH, OW = im2col(input, self.pool_size, self.pool_size, stride=self.stride)
        patches = patches.reshape(-1, C, self.pool_size*self.pool_size)
        self.argmax = patches.argmax(axis=2)

        out = patches.max(axis=2)
        out = out.reshape(N, OH, OW, C).transpose(0,3,1,2)
        return out

    def backward(self, d_out, lr=None):
        N, C, H, W = self.input.shape
        # print(d_out.shape)
        dum1, dum2, OH, OW = d_out.shape

        d_flat = d_out.transpose(0,2,3,1).reshape(-1, C)
        patches_grad = np.zeros((d_flat.shape[0], C, self.pool_size*self.pool_size), dtype=d_flat.dtype)

        i = np.arange(patches_grad.shape[0])[:,None]
        patches_grad[i, np.arange(C), self.argmax] = d_flat

        cols_grad = patches_grad.reshape(-1, C*self.pool_size*self.pool_size)
        return col2im(cols_grad, self.input.shape, self.pool_size, self.pool_size, stride=self.stride)

In [14]:
class FC(Layer):
    def __init__(self, input_size=None, output_size=None):
        self.input_size = input_size
        self.output_size = output_size
        self.weights = None
        self.biases = None

    def forward(self, input):
        self.input = input

        if self.weights is None and self.output_size is not None:
            self.input_size = input.shape[-1]
            # self.weights = np.random.randn(self.output_size, self.input_size)
            limit = np.sqrt(6 / (self.input_size + self.output_size))
            self.weights = np.random.uniform(-limit, limit, (self.output_size, self.input_size))
            self.biases = np.zeros(self.output_size)

        if input.ndim == 1:
            # print("Final FC output before sigmoid:", input[:5].flatten())
            return np.dot(self.weights, input) + self.biases
        else:
            # print("Final FC output before sigmoid:", input[:5].flatten())

            return np.dot(input, self.weights.T) + self.biases

    def backward(self, d_out, lr):
        # print(f"[FC backward] d_out.shape={d_out.shape}, input.shape={self.input.shape}, weights.shape={self.weights.shape}")
        if self.input.ndim == 1 and self.weights is not None:
            weights_gradient = np.outer(d_out, self.input)
            input_gradient = np.dot(self.weights.T, d_out)
        else:
            weights_gradient = np.dot(d_out.T, self.input) / d_out.shape[0]
            input_gradient = np.dot(d_out, self.weights)

        self.weights_grad = weights_gradient
        self.biases_grad = d_out.mean(axis=0) if d_out.ndim > 1 else d_out

        return input_gradient


In [15]:
class Dropout(Layer):
    def __init__(self, dropout_rate=0.5):
        self.rate = dropout_rate
        self.mask = None
        self.training = True

    def forward(self, input):
        if self.training:
            self.mask = (np.random.rand(*input.shape) > self.rate).astype(input.dtype)
            return input * self.mask / (1.0 - self.rate)
        else:
            return input

    def backward(self, d_out, lr=None):
        if self.training:
            return d_out * self.mask / (1.0 - self.rate)
        else:
            return d_out


In [16]:
class Sigmoid(Layer):
    def forward(self, input):
        self.input = np.clip(input, -500, 500)
        self.out = 1 / (1 + np.exp(-self.input))
        return self.out

    def backward(self, d_out, lr=None):
        # print(f"{self.__class__.__name__} received grad of shape {d_out.shape}")
        return d_out * self.out * (1-self.out)

In [17]:
def BCE(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-15, 1-(1e-15))
    return -(y_true*np.log(y_pred) + (1-y_true) * np.log(1-y_pred))

def BCEG(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-15, 1-(1e-15))
    return (y_pred - y_true) / (y_pred * (1 - y_pred))

In [ ]:
import numpy
import cupy as np
import gc

class NumpyCNN:
    def __init__(self):
        self.layers = [
            # Block 1
            Conv2D(kernel_size=3, output_depth=16), ReLU(),
            Conv2D(kernel_size=3, output_depth=16), ReLU(),
            MaxPool2D(pool_size=2, stride=2),
            Dropout(dropout_rate=0.25),

            # Block 2
            Conv2D(kernel_size=3, output_depth=32), ReLU(),
            Conv2D(kernel_size=3, output_depth=32), ReLU(),
            MaxPool2D(pool_size=2, stride=4),
            Dropout(dropout_rate=0.25),

            # Fully connected
            Flatten(),
            FC(input_size=1152, output_size=256), ReLU(),
            Dropout(dropout_rate=0.5),
            FC(input_size=256, output_size=1), Sigmoid()
        ]

    def forward(self, x):
        for i, layer in enumerate(self.layers):
            if isinstance(layer, Dropout):
                layer.training = True

            # print(f"Before layer #{i+1} {layer.__class__.__name__}: Input shape is {x.shape}")
            x = layer.forward(x)
        return x

    def backward(self, grad, lr):
        for layer in reversed(self.layers):
            grad = layer.backward(grad, lr)
        return grad


    def predict(self, X, batch_size=16):
        self.eval()
        preds = []
        for i in range(0, len(X), batch_size):
            x_batch = X[i:i+batch_size]
            x = x_batch
            for layer in self.layers:
                if isinstance(layer, Dropout):
                    layer.training = False
                x = layer.forward(x)
            preds.append(x)

            gc.collect()
            np.get_default_memory_pool().free_all_blocks()
            np.cuda.Device().synchronize()

        return np.concatenate(preds, axis=0)


    def eval(self):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                layer.training = False


    def save(self, filename):
        save_dict = {}
        for i, layer in enumerate(self.layers):
            ps = layer.get_parameters()
            cpu_ps = [np.asnumpy(p) for p in ps]
            for j, p in enumerate(cpu_ps):
                save_dict[f"layer_{i}_param_{j}"] = p
        numpy.savez(filename, **save_dict)
        print(f"Saved model at {filename}")


    def load(self, filename, input_shape):
        dummy_input = np.zeros(input_shape)
        self.predict(dummy_input)

        data = numpy.load(filename, allow_pickle=True)
        for i, layer in enumerate(self.layers):
            ps = []
            j = 0
            while True:
                key = f"layer_{i}_param_{j}"
                if key in data:
                    ps.append(np.array(data[key]))
                    j += 1
                else:
                    break
            if ps:
                layer.set_parameters(ps)


        print(f"Model from {filename} loaded into {type(self).__name__}")


    def collect_parameters(self):
        params = []
        for layer in self.layers:
            params.extend(layer.get_parameters())
        return params

    def collect_gradients(self):
        grads = []
        for layer in self.layers:
            if hasattr(layer, "get_gradients"):
                grads.extend(layer.get_gradients())
        return grads

In [25]:
# import time

def train(model, X_train, y_train, epochs, lr, batch_size, val_split, patience, save_path):
    n = len(X_train)
    split_index = int(n*(1-val_split))

    X_val = X_train[split_index:]
    y_val = y_train[split_index:]
    X_train = X_train[:split_index]
    y_train = y_train[:split_index]
    n_train = len(X_train)

    optimizer = None
    best_val_loss = float('inf')
    best_weights = None
    epochs_without_improvement = 0

    for epoch in range(epochs):
        total_loss = 0.0
        correct = 0


        indices = np.random.permutation(n_train)
        X_train = X_train[indices]
        y_train = y_train[indices]

        for start in range(0, n_train, batch_size):
            # start_time = time.time()
            end = start + batch_size
            batch_x = X_train[start:end]
            batch_y = y_train[start:end].reshape(-1, 1)

            out = model.forward(batch_x)

            if optimizer is None:
                optimizer = Adam(
                    model.collect_parameters(),
                    lr=lr,
                    beta1=0.9,
                    beta2=0.999,
                    epsilon=1e-8
                )

            loss = BCE(batch_y, out).mean()
            grad = BCEG(batch_y, out)

            total_loss += loss * batch_y.shape[0]
            model.backward(grad, lr=None)
            grads = model.collect_gradients()
            optimizer.step(grads)

            gc.collect()
            np.get_default_memory_pool().free_all_blocks()
            np.cuda.Device().synchronize()

            preds = (out > 0.5).astype(int).flatten()
            correct += np.sum(preds == batch_y.flatten())
            # print(time.time()-start_time)

        val_out = model.predict(X_val, batch_size=16)
        val_loss = BCE(y_val.reshape(-1,1), val_out).mean()
        val_preds = (val_out > 0.5).astype(int).flatten()
        val_acc = np.mean(val_preds == y_val.flatten())

        train_acc = correct / n_train
        train_loss = total_loss/n_train
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} — Train Acc: {train_acc:.4f} — Val Loss: {val_loss:.4f} — Val Acc: {val_acc:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_weights = [np.copy(p) for p in model.collect_parameters()]
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}")
                break

    if best_weights:
        param_id = 0
        for layer in model.layers:
            params = layer.get_parameters()
            if params:
                count = len(params)
                layer.set_parameters(best_weights[param_id:param_id+count])
                param_id += count

    if save_path:
        model.save(save_path)


In [ ]:
# np.cuda.runtime.deviceSynchronize()
model = NumpyCNN()
filename = "/content/drive/MyDrive/braai_cnn/models/numpy_cnn.npz"

train(model, X, y, epochs=50, lr=0.001, batch_size=64, val_split=0.05, patience=5, save_path=filename)

Epoch 1/50 - Train Loss: 0.7173 — Train Acc: 0.5699 — Val Loss: 0.6377 — Val Acc: 0.6355
Epoch 2/50 - Train Loss: 0.6169 — Train Acc: 0.6501 — Val Loss: 0.5716 — Val Acc: 0.6786
Epoch 3/50 - Train Loss: 0.5740 — Train Acc: 0.6894 — Val Loss: 0.5651 — Val Acc: 0.6857
Epoch 4/50 - Train Loss: 0.5546 — Train Acc: 0.7046 — Val Loss: 0.5205 — Val Acc: 0.7128
Epoch 5/50 - Train Loss: 0.5390 — Train Acc: 0.7149 — Val Loss: 0.5392 — Val Acc: 0.7067
Epoch 6/50 - Train Loss: 0.5217 — Train Acc: 0.7262 — Val Loss: 0.4868 — Val Acc: 0.7552
Epoch 7/50 - Train Loss: 0.5088 — Train Acc: 0.7367 — Val Loss: 0.5102 — Val Acc: 0.7361
Epoch 8/50 - Train Loss: 0.4957 — Train Acc: 0.7445 — Val Loss: 0.4819 — Val Acc: 0.7495
Epoch 9/50 - Train Loss: 0.4877 — Train Acc: 0.7568 — Val Loss: 0.4523 — Val Acc: 0.7669
Epoch 10/50 - Train Loss: 0.4726 — Train Acc: 0.7671 — Val Loss: 0.4500 — Val Acc: 0.7719
Epoch 11/50 - Train Loss: 0.4644 — Train Acc: 0.7732 — Val Loss: 0.4432 — Val Acc: 0.7813
Epoch 12/50 - Train